# Pamela Santana
# Projet 9
# Produisez une étude de marché - La poule qui chante

In [1]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import scipy.stats as st
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import cluster, metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import subprocess
import geopandas as gpd
from geopy.geocoders import Nominatim


In [2]:
#Importation de fichiers
dispo2017 = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
pop2000_2018 = pd.read_csv('Population_2000_2018.csv')


In [3]:
dispo2017

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.00,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.00,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.00,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.00,S,Données standardisées
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176595,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5142,Nourriture,2899,Miscellanees,2017,2017,Milliers de tonnes,19.00,S,Données standardisées
176596,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,645,Disponibilité alimentaire en quantité (kg/pers...,2899,Miscellanees,2017,2017,kg,1.33,Fc,Donnée calculée
176597,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,664,Disponibilité alimentaire (Kcal/personne/jour),2899,Miscellanees,2017,2017,Kcal/personne/jour,1.00,Fc,Donnée calculée
176598,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,674,Disponibilité de protéines en quantité (g/pers...,2899,Miscellanees,2017,2017,g/personne/jour,0.04,Fc,Donnée calculée


In [4]:
# Supprimer les doublons et les valeurs vides (NaN)
dispo2017 = dispo2017.drop_duplicates().dropna()

# Sélectionner les colonnes pertinentes
dispo2017 = dispo2017 [['Zone', 'Élément', 'Produit', 'Valeur', 'Unité' ]]

# Afficher le DataFrame nettoyé
print(dispo2017)

               Zone                                            Élément  \
0       Afghanistan                                         Production   
1       Afghanistan                            Importations - Quantité   
2       Afghanistan                                 Variation de stock   
3       Afghanistan                            Exportations - Quantité   
4       Afghanistan                           Disponibilité intérieure   
...             ...                                                ...   
176595     Zimbabwe                                         Nourriture   
176596     Zimbabwe  Disponibilité alimentaire en quantité (kg/pers...   
176597     Zimbabwe     Disponibilité alimentaire (Kcal/personne/jour)   
176598     Zimbabwe  Disponibilité de protéines en quantité (g/pers...   
176599     Zimbabwe  Disponibilité de matière grasse en quantité (g...   

                Produit   Valeur               Unité  
0       Blé et produits  4281.00  Milliers de tonnes  
1

In [5]:
dispo2017['Valeur'] = dispo2017['Valeur'].astype(float)
dispo2017['Unité'] = dispo2017['Unité'].astype(str)

In [6]:
dispo2017['Élément'].unique()

array(['Production', 'Importations - Quantité', 'Variation de stock',
       'Exportations - Quantité', 'Disponibilité intérieure',
       'Aliments pour animaux', 'Semences', 'Pertes', 'Résidus',
       'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Traitement', 'Autres utilisations (non alimentaire)',
       'Alimentation pour touristes'], dtype=object)

In [7]:
# Création d'une liste pour ne garder que celles utiles
element = ( 'Production', 'Importations - Quantité', 'Variation de stock',
    'Exportations - Quantité', 'Disponibilité intérieure', 'Nourriture',
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)',)

In [8]:
# Conservation des éléments cités dans la liste pour le df
dispo2017 = dispo2017.loc[dispo2017['Élément'].isin(element), :]

dispo2017

,Zone,Élément,Produit,Valeur,Unité
0,Afghanistan,Production,Blé et produits,4281.00,Milliers de tonnes
1,Afghanistan,Importations - Quantité,Blé et produits,2302.00,Milliers de tonnes
2,Afghanistan,Variation de stock,Blé et produits,-119.00,Milliers de tonnes
3,Afghanistan,Exportations - Quantité,Blé et produits,0.00,Milliers de tonnes
4,Afghanistan,Disponibilité intérieure,Blé et produits,6701.00,Milliers de tonnes
...,...,...,...,...,...
176592,Zimbabwe,Disponibilité intérieure,Miscellanees,19.00,Milliers de tonnes
176595,Zimbabwe,Nourriture,Miscellanees,19.00,Milliers de tonnes
176596,Zimbabwe,Disponibilité alimentaire en quantité (kg/pers...,Miscellanees,1.33,kg
176597,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Miscellanees,1.00,Kcal/personne/jour


In [9]:
# choisir seulement la viande de poule
dispo2017 = dispo2017.loc[dispo2017['Produit'] == 'Viande de Volailles', :]
dispo2017

,Zone,Élément,Produit,Valeur,Unité
651,Afghanistan,Production,Viande de Volailles,28.00,Milliers de tonnes
652,Afghanistan,Importations - Quantité,Viande de Volailles,29.00,Milliers de tonnes
653,Afghanistan,Variation de stock,Viande de Volailles,0.00,Milliers de tonnes
654,Afghanistan,Disponibilité intérieure,Viande de Volailles,57.00,Milliers de tonnes
657,Afghanistan,Nourriture,Viande de Volailles,55.00,Milliers de tonnes
...,...,...,...,...,...
176391,Zimbabwe,Disponibilité intérieure,Viande de Volailles,76.00,Milliers de tonnes
176396,Zimbabwe,Nourriture,Viande de Volailles,67.00,Milliers de tonnes
176397,Zimbabwe,Disponibilité alimentaire en quantité (kg/pers...,Viande de Volailles,4.68,kg
176398,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Viande de Volailles,16.00,Kcal/personne/jour


In [10]:
# choisir le pays avec le plus grande quantité d'exportation viande de volailles (pour les enlever, outliers)
# Filtrar las filas donde 'Élément' sea 'Exportations - Quantité'
exportations_df = dispo2017[dispo2017['Élément'] == 'Exportations - Quantité']

# Agrupar por 'Zone' y sumar los valores de 'Valeur'
exportations_par_pays = exportations_df.groupby('Zone')['Valeur'].sum()

# Convertir la Serie resultante a DataFrame
exportations_par_pays_df = exportations_par_pays.reset_index(name='Total Exportations')

# Ordenar el DataFrame por la columna 'Total Exportations' de mayor a menor
exportations_par_pays_df = exportations_par_pays_df.sort_values(by='Total Exportations', ascending=False)

# Mostrar el DataFrame ordenado
print(exportations_par_pays_df)

                      Zone  Total Exportations
18                  Brésil              4223.0
133  États-Unis d'Amérique              3692.0
93                Pays-Bas              1418.0
95                 Pologne              1025.0
119              Thaïlande               796.0
..                     ...                 ...
96     Polynésie française                 0.0
36               Dominique                 0.0
65                   Kenya                 0.0
101                 Rwanda                 0.0
134           Îles Salomon                 0.0

[135 rows x 2 columns]


In [11]:
# Effacer liste de pays + exportation
exportations_par_pays_df = ['Brésil', "États-Unis d'Amérique", 'Pays-Bas', 'Pologne', 'Thaïlande']

# Filtrar las filas que no pertenecen a los países con mayor exportación
dispo2017 = dispo2017[~dispo2017['Zone'].isin(exportations_par_pays_df)]

# Mostrar el DataFrame filtrado
print(dispo2017)

               Zone                                            Élément  \
651     Afghanistan                                         Production   
652     Afghanistan                            Importations - Quantité   
653     Afghanistan                                 Variation de stock   
654     Afghanistan                           Disponibilité intérieure   
657     Afghanistan                                         Nourriture   
...             ...                                                ...   
176391     Zimbabwe                           Disponibilité intérieure   
176396     Zimbabwe                                         Nourriture   
176397     Zimbabwe  Disponibilité alimentaire en quantité (kg/pers...   
176398     Zimbabwe     Disponibilité alimentaire (Kcal/personne/jour)   
176399     Zimbabwe  Disponibilité de protéines en quantité (g/pers...   

                    Produit  Valeur               Unité  
651     Viande de Volailles   28.00  Milliers de tonn

In [12]:
# reoganisation
dispo2017= pd.pivot(dispo2017, values='Valeur', index=['Zone'], columns=['Élément'])

dispo2017

Élément,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock
Zone,,,,,,,,,
Afghanistan,5.0,1.53,0.54,57.0,NaN,29.0,55.0,28.0,0.0
Afrique du Sud,143.0,35.69,14.11,2118.0,63.0,514.0,2035.0,1667.0,-0.0
Albanie,85.0,16.36,6.26,47.0,0.0,38.0,47.0,13.0,4.0
Algérie,22.0,6.38,1.97,277.0,0.0,2.0,264.0,275.0,0.0
Allemagne,71.0,19.47,7.96,1739.0,646.0,842.0,1609.0,1514.0,-29.0
...,...,...,...,...,...,...,...,...,...
Égypte,50.0,12.96,4.51,1250.0,1.0,110.0,1250.0,1118.0,-23.0
Émirats arabes unis,147.0,43.47,14.80,412.0,94.0,433.0,412.0,48.0,-26.0
Équateur,83.0,19.31,6.15,341.0,0.0,0.0,324.0,340.0,-1.0


In [13]:
pop2000_2018


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2014,2014,1000 personnes,13586.707,X,Sources internationales sûres,NaN
4407,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2015,2015,1000 personnes,13814.629,X,Sources internationales sûres,NaN
4408,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2016,2016,1000 personnes,14030.331,X,Sources internationales sûres,NaN
4409,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,14236.595,X,Sources internationales sûres,NaN


In [14]:
# Sélectionner année 2017
pop2017= pop2000_2018.loc[pop2000_2018['Année'] == 2017]
pop2017

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
17,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,36296.113,X,Sources internationales sûres,NaN
36,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,57009.756,X,Sources internationales sûres,NaN
55,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,2884.169,X,Sources internationales sûres,NaN
74,OA,Séries temporelles annuelles,4,Algérie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,41389.189,X,Sources internationales sûres,NaN
93,OA,Séries temporelles annuelles,79,Allemagne,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,82658.409,X,Sources internationales sûres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,OA,Séries temporelles annuelles,236,Venezuela (République bolivarienne du),511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,29402.484,X,Sources internationales sûres,NaN
4352,OA,Séries temporelles annuelles,237,Viet Nam,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,94600.648,X,Sources internationales sûres,NaN
4371,OA,Séries temporelles annuelles,249,Yémen,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,27834.819,X,Sources internationales sûres,NaN
4390,OA,Séries temporelles annuelles,251,Zambie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,16853.599,X,Sources internationales sûres,NaN


In [15]:
# Sélectionner les colonnes à besoin
pop2017 = pop2017[['Zone', 'Valeur']]

# Afficher le DataFrame nettoyé
print(pop2017)

                                        Zone     Valeur
17                               Afghanistan  36296.113
36                            Afrique du Sud  57009.756
55                                   Albanie   2884.169
74                                   Algérie  41389.189
93                                 Allemagne  82658.409
...                                      ...        ...
4333  Venezuela (République bolivarienne du)  29402.484
4352                                Viet Nam  94600.648
4371                                   Yémen  27834.819
4390                                  Zambie  16853.599
4409                                Zimbabwe  14236.595

[236 rows x 2 columns]


In [16]:
# Rename de la colonne "valeur" pour "population"
pop2017 = pop2017.rename(columns={'Valeur' : 'Population'})
pop2017

,Zone,Population
17,Afghanistan,36296.113
36,Afrique du Sud,57009.756
55,Albanie,2884.169
74,Algérie,41389.189
93,Allemagne,82658.409
...,...,...
4333,Venezuela (République bolivarienne du),29402.484
4352,Viet Nam,94600.648
4371,Yémen,27834.819
4390,Zambie,16853.599


In [17]:
pop2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 17 to 4409
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Zone        236 non-null    object 
 1   Population  236 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.5+ KB


In [18]:
# Changement du dtype float en int
pop2017['Population'] = pop2017['Population'].astype('int64')

In [19]:
# outliers
display(pop2017.sort_values(by='Population', ascending=False))

,Zone,Population
823,"Chine, continentale",1421021
1952,Inde,1338676
1230,États-Unis d'Amérique,325084
1971,Indonésie,264650
3029,Pakistan,207906
...,...,...
2725,Montserrat,4
1781,Îles Falkland (Malvinas),3
4143,Tokélaou,1
2896,Nioué,1


In [20]:
# Effacer liste de pays + pop
popmax = ['Brésil', "États-Unis d'Amérique", 'Pays-Bas', 'Pologne', 'Thaïlande', "Chine, continentale", 'Inde', 'Indonésie', 'Pakistan']


pop2017 = pop2017[~pop2017['Zone'].isin(popmax)]

# Mostrar el DataFrame filtrado
print(pop2017)

                                        Zone  Population
17                               Afghanistan       36296
36                            Afrique du Sud       57009
55                                   Albanie        2884
74                                   Algérie       41389
93                                 Allemagne       82658
...                                      ...         ...
4333  Venezuela (République bolivarienne du)       29402
4352                                Viet Nam       94600
4371                                   Yémen       27834
4390                                  Zambie       16853
4409                                Zimbabwe       14236

[227 rows x 2 columns]


In [21]:
#Supprimer les lignes dupliquées et les valeurs nulles dans dispo2017
dispo2017.drop_duplicates(inplace=True)
dispo2017.dropna(inplace=True)
pop2017.drop_duplicates(inplace=True)
pop2017.dropna(inplace=True)

/var/folders/cy/t20bxd4s1n30wgrjxd5xxwpr0000gn/T/ipykernel_68309/3789472191.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop2017.drop_duplicates(inplace=True)
/var/folders/cy/t20bxd4s1n30wgrjxd5xxwpr0000gn/T/ipykernel_68309/3789472191.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop2017.dropna(inplace=True)


In [22]:
# Vérifier les doublons et les valeurs nulles dans dispo2017 et pop2017
print("Nombre de lignes dupliquées dans dispo2017 :", dispo2017.duplicated().sum())
print("Nombre de valeurs nulles dans pop2017 :", pop2017.isnull().sum())


Nombre de lignes dupliquées dans dispo2017 : 0
Nombre de valeurs nulles dans pop2017 : Zone          0
Population    0
dtype: int64


In [23]:
# Effectuer une jointure interne sur la clé 'Zone'
newbd = dispo2017.merge(pop2017, on='Zone', how='inner')
# (En utilisant une jointure interne (inner join), je conserve que les lignes des deux tables qui ont une correspondance 
# dans la colonne spécifiée (ici, la colonne "Zone").
# Cela signifie que seules les entrées qui existent dans les deux tables seront incluses dans le résultat final.)


# Afficher le résultat de la jointure
print(newbd)

                    Zone  Disponibilité alimentaire (Kcal/personne/jour)  \
0         Afrique du Sud                                           143.0   
1                Albanie                                            85.0   
2                Algérie                                            22.0   
3              Allemagne                                            71.0   
4                 Angola                                            35.0   
..                   ...                                             ...   
120               Zambie                                            11.0   
121               Égypte                                            50.0   
122  Émirats arabes unis                                           147.0   
123             Équateur                                            83.0   
124         Îles Salomon                                            18.0   

     Disponibilité alimentaire en quantité (kg/personne/an)  \
0                       

In [24]:
pays = newbd['Zone'].unique()
pays

array(['Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne', 'Angola',
       'Antigua-et-Barbuda', 'Arabie saoudite', 'Argentine', 'Arménie',
       'Australie', 'Autriche', 'Azerbaïdjan', 'Barbade', 'Belgique',
       'Belize', 'Bolivie (État plurinational de)', 'Bosnie-Herzégovine',
       'Botswana', 'Bulgarie', 'Bélarus', 'Bénin', 'Cameroun', 'Canada',
       'Chili', 'Chine - RAS de Hong-Kong', 'Chine - RAS de Macao',
       'Chine, Taiwan Province de', 'Chypre', 'Colombie', 'Congo',
       'Costa Rica', 'Croatie', "Côte d'Ivoire", 'Danemark', 'Dominique',
       'El Salvador', 'Espagne', 'Estonie', 'Eswatini', 'Fidji',
       'Finlande', 'France', 'Fédération de Russie', 'Gabon', 'Ghana',
       'Grèce', 'Guatemala', 'Guyana', 'Géorgie', 'Honduras', 'Hongrie',
       "Iran (République islamique d')", 'Iraq', 'Irlande', 'Islande',
       'Israël', 'Italie', 'Jamaïque', 'Japon', 'Jordanie', 'Kazakhstan',
       'Kenya', 'Kirghizistan', 'Koweït', 'Lesotho', 'Lettonie', 'Liban',
    

In [25]:
# Crear un objeto geolocalizador
geolocalizador = Nominatim(user_agent="my_geocoder")

# Crear un diccionario para almacenar las coordenadas de cada país
coordenadas_paises = {}

# Iterar sobre la lista de países
for pais in pays:
    # Intentar obtener las coordenadas del país
    try:
        # Obtener las coordenadas geográficas del país
        ubicacion = geolocalizador.geocode(pais)
        
        # Almacenar las coordenadas en el diccionario
        coordenadas_paises[pais] = (ubicacion.latitude, ubicacion.longitude)
    except:
        # Si no se pueden obtener las coordenadas, asignar None
        coordenadas_paises[pais] = None

# Imprimir el diccionario de coordenadas
print(coordenadas_paises)

{'Afrique du Sud': (-28.8166236, 24.991639), 'Albanie': (-32.000111, 18.5557802), 'Algérie': (28.0000272, 2.9999825), 'Allemagne': (49.1478994, -0.3753212), 'Angola': (-11.8775768, 17.5691241), 'Antigua-et-Barbuda': (17.2234721, -61.9554608), 'Arabie saoudite': (25.6242618, 42.3528328), 'Argentine': (-34.9964963, -64.9672817), 'Arménie': (40.7696272, 44.6736646), 'Australie': (-24.7761086, 134.755), 'Autriche': (47.59397, 14.12456), 'Azerbaïdjan': (40.3936294, 47.7872508), 'Barbade': (13.1500331, -59.5250305), 'Belgique': (50.6402809, 4.6667145), 'Belize': (17.1204943, -88.6859028), 'Bolivie (État plurinational de)': (-17.0568696, -64.9912286), 'Bosnie-Herzégovine': (44.3053476, 17.5961467), 'Botswana': (-23.1681782, 24.5928742), 'Bulgarie': (42.6073975, 25.4856617), 'Bélarus': (53.4250605, 27.6971358), 'Bénin': (9.5293472, 2.2584408), 'Cameroun': (4.6125522, 13.1535811), 'Canada': (61.0666922, -107.991707), 'Chili': (-31.7613365, -71.3187697), 'Chine - RAS de Hong-Kong': None, 'Chine 

In [26]:

geolocalizaciones = [coordenadas_paises[pais] for pais in newbd['Zone']]

# Agregar la lista como una nueva columna llamada 'geolocalizacion'
newbd['geolocalizacion'] = geolocalizaciones

# affichage
print(newbd)

                    Zone  Disponibilité alimentaire (Kcal/personne/jour)  \
0         Afrique du Sud                                           143.0   
1                Albanie                                            85.0   
2                Algérie                                            22.0   
3              Allemagne                                            71.0   
4                 Angola                                            35.0   
..                   ...                                             ...   
120               Zambie                                            11.0   
121               Égypte                                            50.0   
122  Émirats arabes unis                                           147.0   
123             Équateur                                            83.0   
124         Îles Salomon                                            18.0   

     Disponibilité alimentaire en quantité (kg/personne/an)  \
0                       

In [27]:
newbd = newbd.dropna(subset=['geolocalizacion'])

# Convertir los valores de 'geolocalizacion' a cadenas de texto
newbd['geolocalizacion'] = newbd['geolocalizacion'].astype(str)

# Dividir la columna 'geolocalizacion' en dos columnas separadas 'Latitude' y 'Longitude'
newbd[['Latitude', 'Longitude']] = newbd['geolocalizacion'].str.strip('()').str.split(',', expand=True)

# Convertir las nuevas columnas a tipo float
newbd[['Latitude', 'Longitude']] = newbd[['Latitude', 'Longitude']].astype(float)

# Eliminar la columna 'geolocalizacion' original si lo deseas
newbd.drop(columns=['geolocalizacion'], inplace=True)

# Verificar el resultado
print(newbd.head())

             Zone  Disponibilité alimentaire (Kcal/personne/jour)  \
0  Afrique du Sud                                           143.0   
1         Albanie                                            85.0   
2         Algérie                                            22.0   
3       Allemagne                                            71.0   
4          Angola                                            35.0   

   Disponibilité alimentaire en quantité (kg/personne/an)  \
0                                              35.69        
1                                              16.36        
2                                               6.38        
3                                              19.47        
4                                              10.56        

   Disponibilité de protéines en quantité (g/personne/jour)  \
0                                              14.11          
1                                               6.26          
2                            

In [28]:
# Vos données actuelles

# Coordonées de la France (latitude et longitude)
latitude_france = 46.603354
longitude_france = 1.888334

# Fonction pour calculer la distance entre chaque pays et la France
def distance_to_france(row):
    return geodesic((latitude_france, longitude_france), (row['Latitude'], row['Longitude'])).kilometers

# Appliquer la fonction à chaque ligne du DataFrame
newbd['Distance_FR'] = newbd.apply(distance_to_france, axis=1)

# Afficher le DataFrame avec la nouvelle colonne

newbd

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock,Population,Latitude,Longitude,Distance_FR
0,Afrique du Sud,143.0,35.69,14.11,2118.0,63.0,514.0,2035.0,1667.0,-0.0,57009,-28.816624,24.991639,8670.232479
1,Albanie,85.0,16.36,6.26,47.0,0.0,38.0,47.0,13.0,4.0,2884,-32.000111,18.555780,8864.865221
2,Algérie,22.0,6.38,1.97,277.0,0.0,2.0,264.0,275.0,0.0,41389,28.000027,2.999983,2067.002227
3,Allemagne,71.0,19.47,7.96,1739.0,646.0,842.0,1609.0,1514.0,-29.0,82658,49.147899,-0.375321,329.688250
4,Angola,35.0,10.56,3.60,319.0,0.0,277.0,315.0,42.0,-0.0,29816,-11.877577,17.569124,6663.383921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Zambie,11.0,3.42,1.15,60.0,1.0,12.0,58.0,49.0,-0.0,16853,-14.518912,27.558988,7240.673367
121,Égypte,50.0,12.96,4.51,1250.0,1.0,110.0,1250.0,1118.0,-23.0,96442,26.254049,29.267547,3304.197529
122,Émirats arabes unis,147.0,43.47,14.80,412.0,94.0,433.0,412.0,48.0,-26.0,9487,24.000249,53.999483,5241.154134
123,Équateur,83.0,19.31,6.15,341.0,0.0,0.0,324.0,340.0,-1.0,16785,0.606015,18.780073,5355.472384


In [29]:
# Enregistrement du DataFrame nettoyé
newbd.to_csv('newbd.csv')